In [45]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [46]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [47]:
peft_model_id = "/home/ysx/src/AI/llm_demo/outputs/checkpoint-1000"
config = PeftConfig.from_pretrained(peft_model_id)


In [48]:

inference_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True, 
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.15s/it]


In [49]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
)

In [50]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [51]:
model = inference_model

In [52]:
model = model.eval()

In [53]:
def generate_prompt(instruction, input):
    return (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    )
   

In [54]:
def make_inference(instruction, input):
  question = generate_prompt(instruction, input)
  batch = tokenizer(question, return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)

  print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [55]:
i = "定义五行的生克制化规律。"

In [56]:
t = "五行的生克制化规律是五行结构系统在正常情况下的自动调节机制。五行之间通过相生相克的关系，维持事物的平衡和发展。相生是指五行之间互相滋生和促进的关系，如木生火、火生土等。相克是指五行之间相互制约的关系，如木克土、土克水等。制化关系是生克关系的结合，生中有克、克中有生，相反相成，维持和促进事物相对平衡协调和发展变化。"

In [57]:
make_inference(i, t)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
定义五行的生克制化规律。

### Input:
五行的生克制化规律是五行结构系统在正常情况下的自动调节机制。五行之间通过相生相克的关系，维持事物的平衡和发展。相生是指五行之间互相滋生和促进的关系，如木生火、火生土等。相克是指五行之间相互制约的关系，如木克土、土克水等。制化关系是生克关系的结合，生中有克、克中有生，相反相成，维持和促进事物相对平衡协调和发展变化。

### Response:
五行的生克制化规律是指五行相生和相克的动态平衡，其中相生规律体现为五行之间的相互滋养和促进，相克规律体现为五行之间的相互制约和克制，制化规律则是生克结合的复杂关系，确保事物的平衡发展。衣、食、住、动、思这五脏之间也遵循这种规律，维持人体的健康平衡。衣指各种服饰；食指各种食物；住指各种居住条件；动指各种活动；思指各种思考和判断。衣食虽不及药物纯，但能补养五脏；居住不同环境，动诸不同事；思考判断，以取决于情志。衣食住动思，五脏之木火土金水矿，以之生克乘侮，以之成之。


In [58]:
# merge base model and lora model as a standalone model.
# merged_model = model.merge_and_unload()